<div class="output_png output_subarea output_execute_result">
<img src=https://nsiripun.github.io/Polaris-II-Model/polariscometlogo.png width="800px"/>
</div>

Real-Time weather downloader based on ECMWF weather forecast for the Polaris WEBSITE. Model downloader separate. Features 3-hourly weather forecasts for 10 days with minimum 0.4° spatial resolution (~44 km). Forecast is in real-time updated every 6 hours through ECMWF dissemination scheme. 

## Create Downloader Grid ##

In [1]:
import math
import pandas as pd
from tqdm import tqdm
import time
import folium
import numpy as np
for i in tqdm(range(1),ncols = 100, desc = 'Loading Terrain'):
    radius = 1000.0 # m - the following code is an approximation that stays reasonably accurate for distances < 100km
    centerLat = 37.421160# latitude of circle center, decimal degrees
    centerLon = 141.032389  # Longitude of circle center, decimal 
    # parameters
    N = 10 # number of discrete sample points to be generated along the circle
    scale = 100
    # generate points
    lat, lon = centerLat, centerLon #center coordinate
    dist, coors = 200000, 15 #meters, num coordinates in each direction

    #Creating the offset grid
    mini, maxi = -dist*coors, dist*coors
    n_coord = coors*2+1
    axis = np.linspace(mini, maxi, n_coord)
    X, Y = np.meshgrid(axis, axis)


    #avation formulate for offsetting the latlong by offset matrices
    R = 6378137 #earth's radius
    dLat = X/R
    dLon = Y/(R*np.cos(np.pi*lat/180))
    latO = lat + dLat * 180/np.pi
    lonO = lon + dLon * 180/np.pi

    #stack x and y latlongs and get (lat,long) format
    output = np.stack([latO, lonO]).transpose(1,2,0)

    circlePoints = []
    nodes = []
    mapnodes = []
    for i in output:
        for x in i:
            circlePoints.append({'lat':x[0],'lon':x[-1]})
    for i in circlePoints:
        u_lon = i['lon']
        u_lat = i['lat']
        nodes.append([u_lon,u_lat])
        mapnodes.append([u_lat,u_lon])
    #print(nodes)

Loading Terrain: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00, 567.95it/s]


In [2]:
import time
import math
import requests
from datetime import datetime
from datetime import datetime,timezone
import metpy
from metpy.calc import wind_components
from metpy.units import units

print('Server Local Time Is:', datetime.now().strftime("%Y-%m-%dT%H:%M"))
print('UTC Time Is:', datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M"))
import math
import json
#Test API Server
response = requests.get("https://api.open-meteo.com/v1/forecast?latitude=37.42&longitude=141.03&hourly=windspeed_10m,winddirection_10m")
if (response.status_code == 200):
    print("Connected to API", flush=True)
    response = response.json()
    # Code here will only run if the request is successful
elif (response.status_code == 404):
    print("API Server Error: 404")
    try:
        response = requests.get('http://api.open-notify.org/astros.json', timeout=5)
        response.raise_for_status()
        # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)

#Download Data

def getuv1(direction,speed):
    v = -abs(speed) * math.cos(direction*(math.pi/180))
    u = -abs(speed) * math.sin(direction*(math.pi/180))
    return [u,v]
def getuv(direction,speed):
    metx = units.Quantity(speed, "m/s")
    metpyout = metpy.calc.wind_components(metx, math.radians(direction))
    return [metpyout[0].magnitude,metpyout[-1].magnitude]


    
timerun = response["hourly"]["time"][:33] #get the first 3 days of forecast
direction = response["hourly"]["winddirection_10m"][:33] #conv to uv operation
speed = response["hourly"]["windspeed_10m"][:33]
apidata = {}
currentsynctime = list(response['hourly']['time'])[0]
time.sleep(0.01)
for i in timerun:
    apidata[i] = []
for p in tqdm(nodes, ncols = 100, desc = f'Syncing Real-Time Data For UTC {currentsynctime} Timestep'):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={p[-1]}&longitude={p[0]}&hourly=windspeed_80m,winddirection_80m")
    response = response.json()
    timerun = response["hourly"]["time"][:33] #get the first 3 days of forecast
    direction = response["hourly"]["winddirection_80m"][:33] #conv to uv operation
    speed = response["hourly"]["windspeed_80m"][:33]
    for i in range(len(timerun)):
        indexer = timerun[i]
        uv = getuv(direction[i],speed[i])
        #print(uv)
        apidata[indexer].append({'u_component_of_wind_10m':uv[0],'v_component_of_wind_10m':uv[-1]})


#print(apidata)

Server Local Time Is: 2022-04-18T17:28
UTC Time Is: 2022-04-19T00:28
Connected to API


Syncing Real-Time Data For UTC 2022-04-19T00:00 Timestep: 100%|███| 961/961 [11:07<00:00,  1.44it/s]


In [3]:
#print(len(apidata[date]))
textfile = open("fukutemp.txt", "w")
skipint = 0
for date in timerun:
    textfile.write(date + "\n")
    for i in range(len(apidata[date])):
        try:
            #print(apidata[date][i])
            #txtwrite = (apidata[date][i].update(apidata[date][i+1]))
            #print(txtwrite)
            textfile.write(str(apidata[date][i]) +'\n')
        except:
            skipint = skipint + 1
        #textfile.write(str(i.update(apidata[date][apidata[date].index(i)+1])) + "\n")
        #print(str(i.update(apidata[date][apidata[date].index(i)+1])) + "\n")
textfile.close()
print('Write Skips: ', skipint)

Write Skips:  0


In [1]:
import ast
import json
import plotly.graph_objs as go
from datetime import datetime,timezone,timedelta
import math
import pandas as pd
from tqdm import tqdm
import time
import folium
import numpy as np
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel to flush iPyleaflet Variables


now_utc = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M")
input_utc = datetime.now(timezone.utc)


def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))


search_utc = hour_rounder(input_utc)
search_utc = search_utc.strftime("%Y-%m-%dT%H:00")
print('Displaying Wind Data For', now_utc, 'UTC')

with open('fukutemp.txt') as f:
    content = f.readlines()
extracted = []

index = [x for x in range(len(content)) if str(search_utc) in content[x]][-1]
print("updated node: ", index)
for i in range(index+1,index+962):
    extracted.append(ast.literal_eval(content[i]))
open('fukuCOMET.txt', 'w').close() 
#print(extracted)

#Re-node GENERATION
for i in tqdm(range(1),ncols = 100, desc = 'Loading Terrain'):
    radius = 1000.0 # m - the following code is an approximation that stays reasonably accurate for distances < 100km
    centerLat = 37.421160# latitude of circle center, decimal degrees
    centerLon = 141.032389  # Longitude of circle center, decimal 
    # parameters
    N = 10 # number of discrete sample points to be generated along the circle
    scale = 100
    # generate points
    lat, lon = centerLat, centerLon #center coordinate
    dist, coors = 200000, 15 #meters, num coordinates in each direction

    #Creating the offset grid
    mini, maxi = -dist*coors, dist*coors
    n_coord = coors*2+1
    axis = np.linspace(mini, maxi, n_coord)
    X, Y = np.meshgrid(axis, axis)


    #avation formulate for offsetting the latlong by offset matrices
    R = 6378137 #earth's radius
    dLat = X/R
    dLon = Y/(R*np.cos(np.pi*lat/180))
    latO = lat + dLat * 180/np.pi
    lonO = lon + dLon * 180/np.pi

    #stack x and y latlongs and get (lat,long) format
    output = np.stack([latO, lonO]).transpose(1,2,0)

    circlePoints = []
    nodes = []
    mapnodes = []
    for i in output:
        for x in i:
            circlePoints.append({'lat':x[0],'lon':x[-1]})
    for i in circlePoints:
        u_lon = i['lon']
        u_lat = i['lat']
        nodes.append([u_lon,u_lat])
        mapnodes.append([u_lat,u_lon])


from ipyleaflet import Map, basemaps
from ipyleaflet.velocity import Velocity
geedata = {'lat':[],'lon':[],'u_wind':[],'v_wind':[]} 


for i in range(len(mapnodes)):
    geedata['lat'].append(mapnodes[i][0])
    geedata['lon'].append(mapnodes[i][-1])
    geedata['u_wind'].append(extracted[i]['u_component_of_wind_10m'])
    geedata['v_wind'].append(extracted[i]['v_component_of_wind_10m'])
df = pd.DataFrame(geedata)
df = df.set_index(['lat', 'lon'])
geedataxr = df.to_xarray()


map12_ = Map(
    center=(37.396113, 140.477330),
    zoom=7,
    interpolation="linear",
    basemap=basemaps.CartoDB.DarkMatter,
)

wind_map = Velocity(
    data=geedataxr,
    zonal_speed="u_wind",
    meridional_speed="v_wind",
    latitude_dimension="lat",
    longitude_dimension="lon",
    velocity_scale=0.001,
    max_velocity= 35,
)

map12_.add_layer(wind_map)
map12_.save("fukuCOMET.txt")
map12_
#clean text file


#print(extracted)


Loading Terrain: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00, 564.97it/s]

Displaying Wind Data For 2022-04-19T00:42 UTC
updated node:  962


Map(center=[37.396113, 140.47733], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Hosting JSON Files to Web Interface ##

In [8]:

import linecache

with open(r'fukuCOMET.txt', 'r') as fp:
    # read line number 3 to 5
    # index starts from 0
    x = fp.readlines()[175:2137]
x[0] =  '[\n' #cleanup
x[-1] = ']\n'
with open('fukuCOMETjson.json', 'w') as f:
    for i in x:
        f.write(i)


*# HTTPRequest in JS #*

In [9]:
import requests
import json
url = 'https://api.jsonbin.io/b/625c241cbc312b30ebe8694a'
headers = {
  'Content-Type': 'application/json',
  'X-Master-Key': '<$2b$10$XzVYIgLc388T1ZmF6jmG..DjmOsA5XjnbkvinDYG9BwdWn8NhFudK>'
}
f = open('fukuCOMETjson.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

req = requests.put(url, json=data, headers=headers)
#print(req.text)
url = 'https://api.jsonbin.io/v3/b/625c241cbc312b30ebe8694a/versions'

headers2 = {
  'X-Master-Key': '$2b$10$XzVYIgLc388T1ZmF6jmG..DjmOsA5XjnbkvinDYG9BwdWn8NhFudK',
  'X-preserve-latest':'true'
} #roll back versioning


req = requests.delete(url, json=None, headers=headers2)
print(req.text)

{"metaData":{"id":"625c241cbc312b30ebe8694a","versionsDeleted":1},"message":"Versions for the Bin are deleted successfully and latest version preserved on the base record."}
